In [1]:
import pandas as pd
import numpy as np 
def join_files(excel_file, excel_sheets, target_dim, target_dim_replace):
    # files
    temp = pd.DataFrame()
    for ele in excel_sheets:
        df = pd.read_excel(excel_file, sheet_name=ele)
        df.columns = df.columns.str.lower()
        df.rename(columns={target_dim: target_dim_replace}, inplace=True)
        temp = temp.append(df, sort=False)
    return temp

In [2]:
import re
import pandas as pd
from bamboo_lib.connectors.models import Connector
from bamboo_lib.models import EasyPipeline, PipelineStep
from bamboo_lib.steps import LoadStep
#from etl.survey_indicators.helpers import join_files
from static import COLUMNS_RENAME
from shared import ReplaceStep

In [3]:
REPLACE_DICT = {
   'Explotación de minas y cante' : 'Explotación de minas y canteras',
   'Suministro de agua, evacuación de aguas residuales, gestión de desechos y descontaminación' : 'Suministro de agua; evacuación de aguas residuales, gestión de desechos y descontaminación',
   'Comercio al por mayor y al por menor, reparación de vehículos automotores y motocicletas' : 'Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas',
   'Actividades profesionales, científicas y técnica' : 'Actividades profesionales, científicas y técnicas',
}


COLUMNS_RENAME = {
    'año': 'year',
    'indicador': 'indicator_id',
    'categoría': 'category_id'
}

SIZE_REPLACE = {
    'Empresa micro': 1,
    'Empresa pequeña': 2,
    'Empresa mediana': 3,
    'Empresa grande': 4
}

COLUMNS_RENAME = {
    'año': 'year',
    'indicador': 'indicator_id',
    'categoría': 'category_id'
}


In [4]:
data = pd.ExcelFile('../../../../datasets/20201007/03. Indicadores estimados DSE - Encuestas (06-10-2020-07-10-2020))/felipe/061020 ENHAT_Indicadores.xlsx')

nation = [x for x in data.sheet_names if re.findall('IND_.*_A', x) != []]
nation.remove('IND_66_A')
industry = [x for x in data.sheet_names if re.findall('IND_.*_C', x) != []]
org_size = [x for x in data.sheet_names if re.findall('IND_.*_B', x) != []]
workforce = ['IND_66_A']

df = join_files(data, nation, 'nivel_desagregación', 'nation_id')
df = df.append(join_files(data, industry, 'nivel_desagregación', 'industry_id'), sort=False)
df = df.append(join_files(data, org_size, 'nivel_desagregación', 'size_id'), sort=False)
df = df.append(join_files(data, workforce, 'nivel_desagregación', 'workforce_id'), sort=False)

In [5]:
df['industry_id'] = df['industry_id'].replace(REPLACE_DICT)

In [6]:
df.isna().sum()

año                0
indicador          0
nation_id       3011
categoría        206
estimate           0
stderr             0
dof                0
t                153
p                153
min95              0
max95              0
coef_var          96
popul_size         0
industry_id      652
size_id         2797
workforce_id    3218
dtype: int64

In [7]:
#df[['industry_id', 'nation_id','industry_id','size_id','workforce_id']] = df[['industry_id', 'nation_id','industry_id','size_id','workforce_id']].fillna(0)

In [8]:
df['nation_id'].unique()

array(['Nacional', nan], dtype=object)

In [9]:
df['industry_id'].unique()

array([nan, 'Agricultura, ganadería, silvicultura y pesca',
       'Explotación de minas y canteras', 'Industrias manufactureras',
       'Suministro de electricidad, gas, vapor y aire acondicionado',
       'Suministro de agua; evacuación de aguas residuales, gestión de desechos y descontaminación',
       'Construcción',
       'Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas',
       'Transporte y almacenamiento',
       'Actividades de alojamiento y de servicio de comidas',
       'Información y comunicaciones',
       'Actividades financieras y de seguros',
       'Actividades inmobiliarias',
       'Actividades profesionales, científicas y técnicas',
       'Actividades de servicios administrativos y de apoyo', 'Enseñanza',
       'Actividades de atención de la salud humana y de asistencia social',
       'Actividades artísticas, de entretenimiento y recreativas',
       'Otras actividades de servicios'], dtype=object)

In [10]:
df['size_id'].unique()

array([nan, 'Empresa pequeña', 'Empresa mediana', 'Empresa grande'],
      dtype=object)

In [11]:
df['workforce_id'].unique()

array([nan, 'Directores y gerentes',
       'Profesionales científicos e intelectuales',
       'Profesionales técnicos', 'Jefes y empleados administrativos',
       'Trabajodores de servicios y vendedores',
       'Obreros, artesanos y electricistas', 'Industriales y conductores',
       'Ocupaciones elementales'], dtype=object)

In [12]:
df1 = pd.read_csv('../../../../datasets/anexos/ISIC_Rev_4_spanish_structure.txt', encoding='latin-1')
        
sections = [
    ['A', 1, 3],
    ['B', 5, 9],
    ['C', 10, 33],
    ['D', 35, 35],
    ['E', 36, 39],
    ['F', 41, 43],
    ['G', 45, 47],
    ['H', 49, 53],
    ['I', 55, 56],
    ['J', 58, 63],
    ['K', 64, 66],
    ['L', 68, 68],
    ['M', 69, 75],
    ['N', 77, 82],
    ['O', 84, 84],
    ['P', 85, 85],
    ['Q', 86, 88],
    ['R', 90, 93],
    ['S', 94, 96],
    ['T', 97, 98],
    ['U', 99, 99]
]

sections = pd.DataFrame(sections, columns=['section_id', 'interval_lower', 'interval_upper'])

df1['group_id'] = df1.apply(lambda x: x['Code'][0:3] if len(x['Code']) == 4 else np.nan, axis=1)
df1['division_id'] = df1.apply(lambda x: x['Code'][0:2] if len(x['Code']) >= 3 else np.nan, axis=1)
df1['section_id'] = df1.apply(lambda x: float(x['division_id']), axis=1)

for section in df1.section_id.unique():
    for level in range(sections.shape[0]):
        if (section >= sections.interval_lower[level]) & (section <= sections.interval_upper[level]):
            df1.section_id.replace(section, str(sections.section_id[level]), inplace=True)
            break

sections_df1 = df1[df1['Code'].str.len() == 1][['Code', 'Title']].copy()
division_df1 = df1[df1['Code'].str.len() == 2][['Code', 'Title']].copy()
group_df1 = df1[df1['Code'].str.len() == 3][['Code', 'Title']].copy()
df1 = df1[df1['Code'].str.len() == 4]

sections_df1.rename(columns={
    'Code': 'section_id',
    'Title': 'section_name'
}, inplace=True)

division_df1.rename(columns={
    'Code': 'division_id',
    'Title': 'division_name'
}, inplace=True)

group_df1.rename(columns={
    'Code': 'group_id',
    'Title': 'group_name'
}, inplace=True)

df1.rename(columns={
    'Code': 'class_id',
    'Title': 'class_name'
}, inplace=True)

df1 = pd.merge(df1, group_df1, on='group_id', how='left')
df1 = pd.merge(df1, division_df1, on='division_id', how='left')
df1 = pd.merge(df1, sections_df1, on='section_id', how='left')

df1 = df1[['section_name', 'division_name', 'group_name', 'class_name', 'section_id', 'division_id', 'group_id', 'class_id']]

additional_df1 = pd.DataFrame(data=[['No determinado', 'No determinado', 'No determinado', 'No determinado', 'Z', '00', '000', '0000']], columns=df1.columns)

df1 = df1.append(additional_df1)


In [13]:
#division_map = {k:v for (k,v) in zip(df["division_name"], df["division_id"])}



In [14]:
#division_map

In [15]:
INDUSTRY_REPLACE = {k:v for (k,v) in zip(df1["section_name"], df1["section_id"])}

In [16]:
## ReplaceStep()

category_dim = dict(zip(df['categoría'].dropna().unique(), range(1, len(df['categoría'].unique()) + 1 )))
df['categoría'].replace(category_dim, inplace=True)

indicator_dim = dict(zip(df['indicador'].dropna().unique(), range(1, len(df['indicador'].unique()) + 1 )))
df['indicador'].replace(indicator_dim, inplace=True)

size_dim = dict(zip(df['size_id'].dropna().unique(), range(1, len(df['size_id'].unique()) + 1 )))
df['size_id'].replace(size_dim, inplace=True)

workforce_dim = dict(zip(df['workforce_id'].dropna().unique(), range(1, len(df['workforce_id'].unique()) + 1 )))
df['workforce_id'].replace(workforce_dim, inplace=True)


df['industry_id'].replace(INDUSTRY_REPLACE, inplace=True)

In [17]:
workforce_dim

{'Directores y gerentes': 1,
 'Profesionales científicos e intelectuales': 2,
 'Profesionales técnicos': 3,
 'Jefes y empleados administrativos': 4,
 'Trabajodores de servicios y vendedores': 5,
 'Obreros, artesanos y electricistas': 6,
 'Industriales y conductores': 7,
 'Ocupaciones elementales': 8}

In [18]:
pd.DataFrame.from_dict(workforce_dim, orient='index').reset_index()

,index,0
0,Directores y gerentes,1
1,Profesionales científicos e intelectuales,2
2,Profesionales técnicos,3
3,Jefes y empleados administrativos,4
4,Trabajodores de servicios y vendedores,5
5,"Obreros, artesanos y electricistas",6
6,Industriales y conductores,7
7,Ocupaciones elementales,8


In [19]:
INDUSTRY_REPLACE

{'Agricultura, ganadería, silvicultura y pesca': 'A',
 'Explotación de minas y canteras': 'B',
 'Industrias manufactureras': 'C',
 'Suministro de electricidad, gas, vapor y aire acondicionado': 'D',
 'Suministro de agua; evacuación de aguas residuales, gestión de desechos y descontaminación': 'E',
 'Construcción': 'F',
 'Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas': 'G',
 'Transporte y almacenamiento': 'H',
 'Actividades de alojamiento y de servicio de comidas': 'I',
 'Información y comunicaciones': 'J',
 'Actividades financieras y de seguros': 'K',
 'Actividades inmobiliarias': 'L',
 'Actividades profesionales, científicas y técnicas': 'M',
 'Actividades de servicios administrativos y de apoyo': 'N',
 'Administración pública y defensa; planes de seguridad social de afiliación obligatoria': 'O',
 'Enseñanza': 'P',
 'Actividades de atención de la salud humana y de asistencia social': 'Q',
 'Actividades artísticas, de entretenimiento y recreat

In [20]:
### 


df = df[['año', 'indicador', 'industry_id','size_id','workforce_id','nation_id','categoría', 'estimate', 'coef_var', 'popul_size']].copy()
df.rename(columns=COLUMNS_RENAME, inplace=True)

df['nation_id'].replace({'Nacional': 'per'}, inplace=True)
df['nation_id'] = df['nation_id'].astype(str)

df[['industry_id', 'nation_id', 'size_id', 'category_id','workforce_id']] = df[['industry_id', 'nation_id', 'size_id', 'category_id','workforce_id']].fillna(0)

df[['year', 'indicator_id',  'size_id', 'category_id','workforce_id', 'estimate', 'coef_var', 'popul_size']] = df[['year', 'indicator_id',  'size_id', 'category_id','workforce_id',  'estimate', 'coef_var', 'popul_size']].astype(float)


In [21]:
df

,year,indicator_id,industry_id,size_id,workforce_id,nation_id,category_id,estimate,coef_var,popul_size
0,2017.0,1.0,0,0.0,0.0,per,1.0,0.962726,0.328233,87159.476562
1,2017.0,1.0,0,0.0,0.0,per,2.0,0.023208,10.646262,2101.146240
2,2017.0,1.0,0,0.0,0.0,per,3.0,0.014065,14.426764,1273.377197
0,2017.0,2.0,0,0.0,0.0,per,4.0,0.069781,6.040935,6317.585449
1,2017.0,2.0,0,0.0,0.0,per,5.0,0.930219,0.453167,84216.414062
...,...,...,...,...,...,...,...,...,...,...
3,2017.0,102.0,0,0.0,4.0,nan,0.0,3.698851,43.804337,746.351990
4,2017.0,102.0,0,0.0,5.0,nan,0.0,2.456897,19.160416,833.922119
5,2017.0,102.0,0,0.0,6.0,nan,0.0,2.531140,25.279764,246.337982
6,2017.0,102.0,0,0.0,7.0,nan,0.0,2.986001,16.187820,551.998047
